In [1]:
import pandas as pd
import torch
import warnings
from utils import preprocess_SemEval14, evaluate_flan
warnings.filterwarnings("ignore")

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_flan = AutoTokenizer.from_pretrained("Shakhovak/flan-t5-base-absa-joint")
model_flan = AutoModelForSeq2SeqLM.from_pretrained("Shakhovak/flan-t5-base-absa-joint")
model_flan.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [4]:
from datasets import load_dataset
import json

raw_datasets = load_dataset("alexcadillon/SemEval2014Task4", "laptops")
augmented_dataset = raw_datasets.map(preprocess_SemEval14)
augmented_dataset

DatasetDict({
    trial: Dataset({
        features: ['sentenceId', 'text', 'aspectTerms', 'aspect_polarities_list', 'aspect_polarities_output', 'aspect_polarity_input'],
        num_rows: 100
    })
    train: Dataset({
        features: ['sentenceId', 'text', 'aspectTerms', 'aspect_polarities_list', 'aspect_polarities_output', 'aspect_polarity_input'],
        num_rows: 3045
    })
    test: Dataset({
        features: ['sentenceId', 'text', 'aspectTerms', 'aspect_polarities_list', 'aspect_polarities_output', 'aspect_polarity_input'],
        num_rows: 800
    })
})

In [5]:
augmented_dataset["trial"][0]

{'sentenceId': '2128',
 'text': 'I liked the aluminum body.',
 'aspectTerms': [{'term': 'aluminum body',
   'polarity': 'positive',
   'from': '12',
   'to': '25'}],
 'aspect_polarities_list': 'aluminum body:positive',
 'aspect_polarities_output': 'Answer: \naluminum body:positive',
 'aspect_polarity_input': 'Determine the aspects (explicit and implicit) and their sentiment of the sentence: \nI liked the aluminum body.\n'}

In [6]:
evaluate_flan(exp_name='laptops14_JOINT-absa-flan-v1',
              trial_dataset = augmented_dataset['trial'], 
              model=model_flan, 
              tokenizer=tokenizer_flan, 
              device=device
)

wandb: Currently logged in as: katya_shakhova (shakhova). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.4
wandb: Run data is saved locally in /home/jupyter/work/resources/wandb/run-20240312_193725-r5ud11fb
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run laptops14_JOINT-absa-flan-v1
wandb: ⭐️ View project at https://wandb.ai/shakhova/absa_research2
wandb: 🚀 View run at https://wandb.ai/shakhova/absa_research2/runs/r5ud11fb


  0%|          | 0/100 [00:00<?, ?it/s]

wandb:                                                                                
wandb: 
wandb: Run history:
wandb: f1_aspect_ABSA ▁
wandb:  f1_macro_ABSA ▁
wandb:  f1_micro_ABSA ▁
wandb:   f1_sent_ABSA ▁
wandb: 
wandb: 🚀 View run laptops14_JOINT-absa-flan-v1 at: https://wandb.ai/shakhova/absa_research2/runs/r5ud11fb
wandb: Synced 4 W&B file(s), 1 media file(s), 1 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240312_193725-r5ud11fb/logs
